In [1]:
import pandas as pd
import xarray as xr
import numpy as np
from datetime import datetime
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.patches as mpatches
%matplotlib inline
from mpl_toolkits.basemap import Basemap

In [2]:
filename = '/home/duarte/Documentos/mestrado-INPE/aulas/termodinamica/lista_3/cloud.nc'
ds = xr.open_dataset(filename)

In [3]:
ds

<xarray.Dataset> Size: 3GB
Dimensions:    (longitude: 1440, latitude: 721, time: 144)
Coordinates:
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 1kB 2015-04-07 ... 2015-04-12T23:00:00
Data variables:
    hcc        (time, latitude, longitude) float32 598MB ...
    lcc        (time, latitude, longitude) float32 598MB ...
    mcc        (time, latitude, longitude) float32 598MB ...
    tcc        (time, latitude, longitude) float32 598MB ...
    tp         (time, latitude, longitude) float32 598MB ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-03 02:54:20 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [4]:
# Define x, y e T
time = ds.time
time = pd.to_datetime (time.data)
lat = ds.latitude
lon = ds.longitude

In [5]:
def calcular_estatisticas(ds):
    # Extrair variáveis
    hcc = ds['hcc']
    lcc = ds['lcc']
    mcc = ds['mcc']
    tcc = ds['tcc']
    tp = ds['tp']
    
    # Calcular estatísticas
    mean_hcc = np.mean(hcc, axis=0)
    mean_lcc = np.mean(lcc, axis=0)
    mean_mcc = np.mean(mcc, axis=0)
    mean_tcc = np.mean(tcc, axis=0)
    sum_tp = np.sum(tp, axis=0)

    return mean_hcc, mean_lcc, mean_mcc, mean_tcc, sum_tp

mean_hcc, mean_lcc, mean_mcc, mean_tcc, sum_tp = calcular_estatisticas(ds)

In [ ]:
import os

# Caminho para o diretório específico
dirin = "/content/drive/MyDrive/Colab Notebooks/Termodinâmica - Lista 03/"

# Definindo às variáveis
variables = ['hcc', 'lcc', 'mcc', 'tcc']
names = ['High cloud cover', 'Low cloud cover', 'Medium cloud cover', 'Total Cloud Cover']
dates = ['2015-04-08', '2015-04-09', '2015-04-10', '2015-04-11', '2015-04-12']

# Define as linhas no mapa, para todo o globo /:
meridiano = np.arange(10., 351., 60.)
paralelos = np.arange(-90., 91, 30.)

# Loop para entre as variaveis e geração do plot 🔃
for var_i, var in enumerate(variables):
    mean_diaria = ds[var].resample(time='D').mean()
    os.mkdir(os.path.join(dirin, var))  # gera um dir. para cada var.
    for date in dates:
        fig, ax = plt.subplots(figsize=(11, 7))
        plt.title(f'{names[var_i]} - {date}')  # .upper > usado para deixar em caixa ALTA
        m = Basemap(projection='robin', lon_0=180)
        m.drawcoastlines()
        x, y = np.meshgrid(lon, lat)
        px, py = m(x, y)

        fig = m.contourf(px, py, mean_diaria.sel(time=date), np.arange(0, 1.0 + .1, 0.1), extend='both', cmap='jet')
        cbar = m.colorbar(fig, location='right', pad="10%")
        cbar.set_label('Fraction of cloud cover')

        m.drawparallels(paralelos, labels=[False, True, True, False])
        m.drawmeridians(meridiano, labels=[True, False, False, True])
        plt.tight_layout()
        plt.savefig(os.path.join(dirin, var, f'{var}_{date}.png'), bbox_inches='tight', dpi=300)
        # plt.show()
